<a href="https://colab.research.google.com/github/ACCMouli/chandu/blob/main/lllms/analysis/2_summarization_from_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install OpenAI
!pip install langchain
!pip install langchain_community
!pip install Cohere
!pip install langchain-openai langchain-cohere python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 96.0 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.1
    Uninstalling httpx-sse-0.4.1:
      Successfully uninstalled httpx-sse-0.4.1
   ━━━━━━

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata

In [3]:
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

In [4]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI


In [5]:



CSV_PATH = "https://raw.githubusercontent.com/giridhar276/genai/refs/heads/main/datasets/Bank_Customer_conversations.csv"
TEXT_COL = "customer_text"
MODEL = "gpt-4o-mini"
TEMPERATURE = 0
TIMEOUT = 60

OUTPUT_PATH = CSV_PATH.replace(".csv", "_with_summary.csv")

assert os.getenv("OPENAI_API_KEY"), "Please set OPENAI_API_KEY in your environment."


In [6]:

PROMPT = (
    "Summarize the customer's message in ONE clear sentence focusing on intent/issue and requested action. "
    "Do not add details that are not present. "
    "Return only the summary text on a single line.\n\n"
    "CUSTOMER TEXT:\n"
    '\"\"\"{text}\"\"\"\n'
    "Summary:"
)


In [7]:

df = pd.read_csv(CSV_PATH)
df = df[[TEXT_COL]].copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()

print("Rows:", len(df))
df.head(3)


Rows: 30


,customer_text
0,Ms. Patel: My card ending 0530 got blocked aft...
1,Ms. Patel: I filed a loan application 27-Aug-2...
2,Mr. Singh: I filed a loan application 30-Aug-2...


In [8]:

llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE, timeout=TIMEOUT)

def summarize_one(text) :
    return llm.invoke(PROMPT.format(text=text)).content.strip()


In [9]:

summaries = []
for txt in df[TEXT_COL].tolist():
    summaries.append(summarize_one(txt))

df["summary"] = summaries


In [10]:

df.to_csv("summary.csv", index=False)
print(f"Saved: {OUTPUT_PATH}")
df[[TEXT_COL, "summary"]].head(10)


Saved: https://raw.githubusercontent.com/giridhar276/genai/refs/heads/main/datasets/Bank_Customer_conversations_with_summary.csv


,customer_text,summary
0,Ms. Patel: My card ending 0530 got blocked aft...,The customer requests assistance to unblock th...
1,Ms. Patel: I filed a loan application 27-Aug-2...,The customer is requesting the current approva...
2,Mr. Singh: I filed a loan application 30-Aug-2...,The customer is inquiring about the approval s...
3,Mr. Verma: I filed a loan application 10-Sep-2...,The customer is requesting the current approva...
4,Ms. Patel: I submitted an insurance claim 22-A...,The customer is inquiring about the status and...
5,Mr. Khan: I’m considering a term insurance pla...,The customer is requesting information on term...
6,Mr. Khan: I’m considering a term insurance pla...,"The customer is requesting options, premium de..."
7,Mr. Iyer: My card ending 6566 got blocked afte...,The customer requests assistance to unblock th...
8,Mr. Singh: I submitted an insurance claim 12-S...,The customer is inquiring about the status and...
9,Ms. Chawla: I filed a loan application 30-Aug-...,The customer is requesting the current approva...
